# Cálculo de deducciones incrementadas de Tramo 1 y Tramo 2

## A. Importación de módulos y librerías

In [15]:
import pandas as pd
import numpy as np

from remuneraciones.prorrateos import *
from parametrizacion.conceptos import ParamConceptos
from deducciones.incremental.tablas_tramo2 import *

from helpers.bases import tabla
from helpers.progressbar import ProgressBar
from helpers.dict import unpack

from data import CWTR_SUM, DOTACION, CUILES, F572_WEB

## B. Parametrización de conceptos

In [16]:
conceptos = ParamConceptos()
conceptos

## C. Deducción incrementada Tramo 1

### 1. Unión de los orígenes de datos

#### CWTR

In [17]:
cwtr = CWTR_SUM[['legajo', 'CCn', 'mes', 'importe']].copy()
cwtr.rename(columns={'CCn': 'codigo'}, inplace=True)

cwtr

,legajo,codigo,mes,importe
0,6000002,/111,1,323617.99
1,6000002,/111,2,1404173.92
2,6000002,/111,3,351100.00
3,6000002,/111,4,353440.65
4,6000002,/111,5,351100.00
...,...,...,...,...
314786,6114237,/361,12,1777.31
314787,6114237,/365,12,29.59
314788,6114237,/S02,12,986.41
314789,6114237,3003,12,602.30


#### F.572 Web

In [18]:
f572 = F572_WEB.copy()
f572['cuil'] = f572['cuil'].astype('int64')
f572 = f572.merge(CUILES, how='left', on='cuil')
f572 = f572[['legajo', 'codigo', 'mes', 'importe']]
f572.dropna(inplace=True)

f572

,legajo,codigo,mes,importe
0,6106059.0,Q-Deduc-1,7,4380.80
1,6106059.0,Q-Deduc-2,13,3962.89
2,6106059.0,Q-Deduc-22,12,18000.00
3,6071357.0,Q-RetPerPagCu-27,13,113153.25
4,6071357.0,Q-Deduc-22,1,8400.00
...,...,...,...,...
77554,6002316.0,Q-Deduc-1,8,15992.93
77555,6002316.0,Q-Deduc-1,9,11015.22
77556,6002316.0,Q-Deduc-1,10,15526.32
77557,6002316.0,Q-Deduc-1,11,15606.62


#### Prorrateos

In [19]:
tramos = tabla('bases_tramos')
legajos_meses_tramo1 = tramos[tramos['tramo'].eq(1)]
prorrateados = Prorrateos().prorrateados()

prorrateos_aplicables = prorrateados.merge(legajos_meses_tramo1, how='inner', left_on=['legajo', 'mes_pago'], right_on=['legajo', 'mes'])
prorrateos = prorrateos_aplicables[['legajo', 'CCn', 'mes_x', 'importe_x']].copy()
prorrateos.rename(columns={'CCn': 'codigo', 'mes_x': 'mes', 'importe_x': 'importe'}, inplace=True)

prorrateos

,legajo,codigo,mes,importe
0,6000009,#1256,6,713.195714
1,6000009,#1256,7,713.195714
2,6000009,#1256,8,713.195714
3,6000009,#1256,9,713.195714
4,6000009,#1256,10,713.195714
...,...,...,...,...
40110,6106947,#1256,12,706.000000
40111,6106947,#1258,10,0.000000
40112,6106947,#1258,11,0.000000
40113,6106947,#1258,12,0.000000


#### Deducciones de Ley

In [20]:
minimo_no_imponible = {
    'normal': {
        'mni': 252_564.84,
        'esp': 1_212_311.24
    },
    'especial': {
        'mni': 308_129.12,
        'esp': 1_479_019.72
    }
}

provincias_especiales = [
    'Neuquén',
    'Chubut',
    'Río Negro',
    'Tierra del Fuego',
    'Santa Cruz',
    'La Pampa',
]

In [37]:
from data import CWTR

cwtr_provincias = CWTR[['legajo', 'mes', 'provincia']].drop_duplicates().reset_index()

prov_dotacion = list(cwtr_provincias['provincia'].unique())

for p in provincias_especiales:
    if p not in prov_dotacion:
        raise NotImplementedError(f'No hay empleados en la provincia: {p}')


cwtr_provincias

,index,legajo,mes,provincia
0,0,6000002,4,Córdoba
1,1,6000002,6,Córdoba
2,2,6000002,8,Córdoba
3,3,6000002,1,Córdoba
4,5,6000002,2,Córdoba
...,...,...,...,...
32835,227168,6114229,12,Tierra del Fuego
32836,227171,6114231,12,Tierra del Fuego
32837,227174,6114233,12,Tierra del Fuego
32838,227177,6114235,12,Chubut


In [38]:
deducciones_ley = []

for i, row in cwtr_provincias.iterrows():

    legajo, mes, provincia = unpack(row, 'legajo', 'mes', 'provincia')

    tipo_mni = 'especial' if provincia in provincias_especiales else 'normal'
    deducciones = minimo_no_imponible[tipo_mni]

    deducciones_ley.append({'legajo': legajo, 'codigo': 'DedLey-MNI', 'mes': mes, 'importe': deducciones['mni'] / 12 })
    deducciones_ley.append({'legajo': legajo, 'codigo': 'DedLey-Especial', 'mes': mes, 'importe': deducciones['esp'] / 12 })

deducciones_ley = pd.DataFrame.from_records(deducciones_ley)
deducciones_ley

,legajo,codigo,mes,importe
0,6000002,DedLey-MNI,4,21047.070000
1,6000002,DedLey-Especial,4,101025.936667
2,6000002,DedLey-MNI,6,21047.070000
3,6000002,DedLey-Especial,6,101025.936667
4,6000002,DedLey-MNI,8,21047.070000
...,...,...,...,...
65675,6114233,DedLey-Especial,12,123251.643333
65676,6114235,DedLey-MNI,12,25677.426667
65677,6114235,DedLey-Especial,12,123251.643333
65678,6114237,DedLey-MNI,12,25677.426667


#### Unión

In [41]:
total_conceptos = pd.concat([cwtr, f572, prorrateos, deducciones_ley])
assert len(cwtr) + len(f572) + len(prorrateos) + len(deducciones_ley) == len(total_conceptos), 'No coinciden las sumas!'

total_conceptos

,legajo,codigo,mes,importe
0,6000002.0,/111,1,3.236180e+05
1,6000002.0,/111,2,1.404174e+06
2,6000002.0,/111,3,3.511000e+05
3,6000002.0,/111,4,3.534407e+05
4,6000002.0,/111,5,3.511000e+05
...,...,...,...,...
65675,6114233.0,DedLey-Especial,12,1.232516e+05
65676,6114235.0,DedLey-MNI,12,2.567743e+04
65677,6114235.0,DedLey-Especial,12,1.232516e+05
65678,6114237.0,DedLey-MNI,12,2.567743e+04


### 2. Filtrado de conceptos aplicables

In [42]:
# Control
conceptos_parametrizados = list(conceptos.data['CCn'].unique())

for c in list(total_conceptos['codigo'].unique()):
    if c not in conceptos_parametrizados:
        raise NotImplementedError(f'No se ha definido parametría para el concepto: {c}')

In [43]:
df = total_conceptos
df['legajo'] = df['legajo'].astype('int64')

df['Suma'] = np.where(df['codigo'].isin(conceptos.listar('DedIncT1', '+')), df['importe'], 0)
df['Resta'] = np.where(df['codigo'].isin(conceptos.listar('DedIncT1', '-')), df['importe'], 0)
df['DeduccIncremT1'] = df['Suma'] - df['Resta']

# Se filtran cuando sólo sean tramo 1
df = df.merge(legajos_meses_tramo1, how='inner', on=['legajo', 'mes'])

In [44]:
deduc_incr_tramo1_mensual = df.groupby(['legajo','mes'])['DeduccIncremT1'].sum().reset_index()
deduc_incr_tramo1_anual = df.groupby(['legajo'])['DeduccIncremT1'].sum().reset_index()

deduc_incr_tramo1_mensual.to_excel('prueba.xlsx')

## D. Deducción incrementada Tramo 2

In [45]:
legajos_meses_tramo2 = tramos[tramos['tramo'].eq(2)]
legajos_meses_tramo2

,legajo,mes,importe,promedio,base_tramo,tramo
15,6000009.0,4.0,230493.70,232092.88,230493.70,2
16,6000009.0,5.0,246092.17,234892.74,234892.74,2
18,6000009.0,7.0,298932.97,283397.79,283397.79,2
19,6000009.0,8.0,351847.36,306214.31,306214.31,2
22,6000009.0,11.0,429796.04,429796.04,429796.04,2
...,...,...,...,...,...,...
35831,6111355.0,12.0,525739.87,360098.82,360098.82,2
35855,6111361.0,12.0,358571.11,337882.85,337882.85,2
35939,6111382.0,12.0,502104.76,332666.33,332666.33,2
35951,6111383.0,12.0,586905.51,390407.44,390407.44,2


In [46]:
tabla('tablas_afip_tramo2')

,base,deduccion,ini,fin
0,226087,64737.0,1,5
1,226238,64180.0,1,5
2,226389,63669.0,1,5
3,226539,63187.0,1,5
4,226690,62726.0,1,5
...,...,...,...,...
687,429882,2534.0,11,12
688,430321,2005.0,11,12
689,430761,1476.0,11,12
690,431201,947.0,11,12


In [47]:
deducc_tramo2 = []

p = ProgressBar(len(legajos_meses_tramo2))
for i, row in legajos_meses_tramo2.iterrows():
    
    legajo, base, mes = unpack(row, 'legajo', 'base_tramo', 'mes')

    deduccion = TablasTramo2.deduccion(mes, base)
    deducc_tramo2.append({
        'legajo':           legajo, 
        'mes':              mes, 
        'DeduccIncremT2':   deduccion
    })
    
    p.next()

deducc_tramo2 = pd.DataFrame.from_records(deducc_tramo2)
deducc_tramo2

|█████████████████████████████████████████████████░| 98.01 %


,legajo,mes,DeduccIncremT2
0,6000009.0,4.0,53051.00
1,6000009.0,5.0,43997.00
2,6000009.0,7.0,100180.08
3,6000009.0,8.0,39686.27
4,6000009.0,11.0,2534.00
...,...,...,...
6624,6111355.0,12.0,85812.00
6625,6111361.0,12.0,116153.00
6626,6111382.0,12.0,123863.00
6627,6111383.0,12.0,37870.00


## E. Unificación con base tramos

In [48]:
con_tramo1 = tramos.merge(deduc_incr_tramo1_mensual, how='left', on=['legajo', 'mes'])
con_tramo2 = con_tramo1.merge(deducc_tramo2, how='left', on=['legajo', 'mes'])

In [49]:
con_tramo2.to_excel('deducciones_incrementadas.xlsx')